In [231]:
from connection import * 
import numpy as np
from sqlalchemy import create_engine 
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings('ignore')

DB_HOST = DB_HOST
DB_NAME = DB_NAME
DB_USER = DB_USER       
DB_PASS = DB_PASS       
DB_PORT = DB_PORT              

engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

def get_boolean_columns(engine, schema, table_name):
    """Get the right schema and table name for the boolean columns."""
    query = f"""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_schema = '{schema}' AND table_name = '{table_name}' AND data_type = 'boolean';
            """
    result = pd.read_sql(query, engine)
    return result['column_name'].tolist()

clinical = pd.read_sql('SELECT * FROM project.clinical', engine)
pain = pd.read_sql('SELECT * FROM project.pain', engine)
patient = pd.read_sql('SELECT * FROM project.patient', engine)
psychological = pd.read_sql('SELECT * FROM project.psychological', engine)
radvice = pd.read_sql('SELECT * FROM project.radvice', engine)
work = pd.read_sql('SELECT * FROM project.work', engine)

In [232]:
def convert_bool_to_int(table, engine, table_name):   
    """Get Boolean columns from PostgreSQL and convert them to int64 to overcome with NANs""" 
    bool_cols = get_boolean_columns(engine, 'project', table_name)
    for col in bool_cols:
        table[col] = table[col].astype('Int64')
    return table

clinical = convert_bool_to_int(clinical, engine, 'clinical')
pain = convert_bool_to_int(pain, engine, 'pain')
patient = convert_bool_to_int(patient, engine, 'patient')
psychological = convert_bool_to_int(psychological, engine, 'psychological')
radvice = convert_bool_to_int(radvice, engine, 'radvice')
work = convert_bool_to_int(work, engine, 'work')
#patient.serious_disease.value_counts(dropna=False)



In [233]:
# Join all the tables
df_merged = radvice.merge(patient, left_on='patient_id', right_on='patient_id', how='left')
df_merged = df_merged.merge(clinical, left_on='patient_id', right_on='patient_id', how='left')
df_merged = df_merged.merge(psychological, left_on='patient_id', right_on='patient_id', how='left')
df_merged = df_merged.merge(pain, left_on='patient_id', right_on='patient_id', how='left')
df_merged = df_merged.merge(work, left_on='patient_id', right_on='patient_id', how='left')
df_merged = df_merged.drop(columns=['patient_id','treatment_description'])
print("Check final size of df: ", len(df_merged))

df = df_merged.copy()

# continous
continous_columns =  [
    'decreased_mobility',
    'weightloss_per_year',
    'extremely_nervous',
    'irrational_thoughts_risk_lasting',
    'coping_strategy',
    'kinesiophobia_physical_exercise',
    'kinesiophobia_pain_stop',
    'nocturnal_pain',
    'continuous_pain',
    'duration_of_pain',
    'neck_pain_intensity',
    'low_back_pain_intensity',
    'arm_left_pain_intensity',
    'arm_right_pain_intensity',
    'leg_left_pain_intensity',
    'leg_right_pain_intensity'
]
df[continous_columns] = df[continous_columns].astype(float)

cat_cols = [col for col in df.columns if str(df[col].dtype).lower().startswith('int')]
print("Categorical features (Int64):", cat_cols)

Check final size of df:  1527
Categorical features (Int64): ['treatment', 'family_history', 'serious_disease', 'fever', 'uses_analgesics', 'uses_corticosteroids', 'neurogenic_signals', 'loss_muscle_strength', 'failure_symptoms', 'depression', 'stress', 'sick_leave', 'earlier_hospitalization', 'paidwork']


In [234]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Convert age to numerical
df['age'] = df['age'].map({
    '0-19': 10, '20-29': 25, '30-39': 35, '40-49': 45,
    '50-59': 55, '60-69': 65, '70-79': 75, '>=80': 85
}).astype(float)

# Separate features (X) and target (y).
X, y = df.drop(columns=['treatment']), df['treatment']
column_names = X.columns
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
#print("Missing before:", np.isnan(X).sum())

# Custom transformer for feature engineering.
class CustomFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Convert array back to DataFrame so we can manipulate columns by name
        df = pd.DataFrame(X, columns=self.column_names)
        
        # Arm, leg, neck
        df['arm_pain'] = ((df['arm_left_pain_intensity'] > 0) | (df['arm_right_pain_intensity'] > 0)).astype('Int64')
        df['leg_pain'] = ((df['leg_left_pain_intensity'] > 0) | (df['leg_right_pain_intensity'] > 0)).astype('Int64')
        df['neck_pain'] = (df['neck_pain_intensity'] > 0).astype('Int64')

        # Kinesiophobia
        df['kinesiophobia'] = df[['kinesiophobia_pain_stop','kinesiophobia_physical_exercise']].mean(axis=1)

        df = df.drop(columns=['kinesiophobia_pain_stop','kinesiophobia_physical_exercise',
                              'arm_left_pain_intensity','arm_right_pain_intensity',
                                'leg_left_pain_intensity','leg_right_pain_intensity',
                                'neck_pain_intensity',
                                ])
        
        return df
    
fe = CustomFeatureEngineer(column_names=column_names)
X_fe = fe.fit_transform(X)
X_fe_df = pd.DataFrame(X_fe, columns=fe.transform(pd.DataFrame(X, columns=column_names)).columns)
print("Shape of X:", X_fe_df.shape)


Shape of X: (1527, 30)
Shape of y: (1527,)
Shape of X: (1527, 27)


# Modeling

In [235]:
#X_fe = pipeline.named_steps['feature_engineering'].transform(X_train)
#cat_cols = [i for i, dt in enumerate(X_fe.dtypes) if dt.name == 'Int64']
print("Categorical feature indices for SMOTENC:", cat_cols)
cat_cols_idx =[1, 2, 3, 4, 5, 6, 9, 10, 11, 13, 20, 21, 22, 23, 24, 25]

X_train, X_test, y_train, y_test = train_test_split(
    X_fe_df, y, test_size=0.2, stratify=y, random_state=42
)

Categorical feature indices for SMOTENC: ['treatment', 'family_history', 'serious_disease', 'fever', 'uses_analgesics', 'uses_corticosteroids', 'neurogenic_signals', 'loss_muscle_strength', 'failure_symptoms', 'depression', 'stress', 'sick_leave', 'earlier_hospitalization', 'paidwork']


In [243]:
over = SMOTE(sampling_strategy='not majority', random_state=42)
over2 = SMOTENC(categorical_features=cat_cols_idx, sampling_strategy='all', random_state=42)
under = RandomUnderSampler(sampling_strategy='not minority', random_state=42)

pipeline = ImbPipeline(steps=[
    ('imputer', IterativeImputer(max_iter=50, skip_complete=True)),
    #('smote', over),  # activate for oversampling
    ##('undersample', under),   # activate for undersampling
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(class_weight='balanced',n_estimators = 500, random_state = 42))
])

pipeline.fit(X_train, y_train)
score = pipeline.score(X_test, y_test)
print(f"Test score: {score:.4f}")
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred,))


# STRATIFIED cross-validation
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y, cv=cv)
print(f"Mean CV accuracy:: {scores.mean():.4f}")

Test score: 0.5000
              precision    recall  f1-score   support

           1       0.52      0.61      0.56       131
           2       0.00      0.00      0.00        32
           3       0.00      0.00      0.00        12
           4       0.00      0.00      0.00         3
           5       0.48      0.57      0.52       128

    accuracy                           0.50       306
   macro avg       0.20      0.24      0.22       306
weighted avg       0.42      0.50      0.46       306

Mean CV accuracy:: 0.4656


# Pipeline with PCA

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline as ImbPipeline

pipeline = ImbPipeline(steps=[
    ('imputer', IterativeImputer(max_iter=50)),
    #('smote', over),  # activate for oversampling
    ##('undersample', under),   # activate for undersampling
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)),
    ('classifier', RandomForestClassifier(
        class_weight='balanced',
        n_estimators=500,
        random_state=42
    ))
])

pipeline.fit(X_train, y_train)

test_score = pipeline.score(X_test, y_test)
print(f"Random Forest on PCA features - Test score: {test_score:.4f}")

y_pred = pipeline.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Random Forest on PCA features - Test score: 0.4673
Classification Report:
              precision    recall  f1-score   support

           1       0.48      0.59      0.53       131
           2       0.50      0.06      0.11        32
           3       0.00      0.00      0.00        12
           4       0.00      0.00      0.00         3
           5       0.45      0.50      0.48       128

    accuracy                           0.47       306
   macro avg       0.29      0.23      0.22       306
weighted avg       0.45      0.47      0.44       306



In [ ]:
# Create pipeline up to PCA
pipeline_pca = ImbPipeline(steps=[
    ('imputer', IterativeImputer(max_iter=50)),
    #('smote', over),  # activate for oversampling
    ##('undersample', under),   # activate for undersampling
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=4))
])

X_train_pca = pipeline_pca.fit_transform(X_train, y_train)
X_test_pca = pipeline_pca.transform(X_test)

# Train classifier separately
clf_pca = RandomForestClassifier(class_weight='balanced', n_estimators=500, random_state=42)
clf_pca.fit(X_train_pca, y_train)
test_score = clf_pca.score(X_test_pca, y_test)
print(f"Random Forest on PCA features - Test score: {test_score:.4f}")

Random Forest on PCA features - Test score: 0.4641


# Recursive feature elimination


# Feature Selection with Univariate Statistical Tests

In [247]:
#Feature Selection with Univariate Statistical Tests
from sklearn.feature_selection import SelectKBest, f_classif

pipeline_kbest = ImbPipeline(steps=[
    ('imputer', IterativeImputer(max_iter=50, skip_complete=True)),
    #('smote', over),  # activate for oversampling
    ##('undersample', under),   # activate for undersampling
    ('scaler', StandardScaler()),
    ('select', SelectKBest(score_func=f_classif, k=10)),  # <-- Select top 10 features
    ('classifier', RandomForestClassifier(class_weight='balanced',n_estimators = 500, random_state = 42))
])
pipeline_kbest.fit(X_train, y_train)
score_kbest = pipeline_kbest.score(X_test, y_test)
print("Test score with SelectKBest:", score_kbest)

cv_scores_kbest = cross_val_score(pipeline_kbest, X, y, cv=5)
print("Mean CV score with SelectKBest:", cv_scores_kbest.mean())
selector = pipeline_kbest.named_steps['select']
selected_mask = selector.get_support()


Test score with SelectKBest: 0.3790849673202614
Mean CV score with SelectKBest: 0.37524483017250615


In [203]:
# Step 1: Run feature engineering manually
fe = pipeline_kbest.named_steps['feature_engineering']
X_fe = fe.transform(X_train)  # shape: (n_samples, n_features_after_FE)

# Step 2: Build final feature names list that matches the output exactly
base_names = list(fe.column_names)

# Drop columns that are removed in transform()
to_drop = [
    'kinesiophobia_pain_stop',
    'kinesiophobia_physical_exercise',
    'arm_left_pain_intensity',
    'arm_right_pain_intensity',
    'leg_left_pain_intensity',
    'leg_right_pain_intensity',
    'neck_pain_intensity',
]
final_names = [name for name in base_names if name not in to_drop]

# Add engineered features (in the exact order from your transformer!)
final_names += ['arm_pain', 'leg_pain', 'neck_pain', 'kinesiophobia']

# You also converted 'age' to a numeric, but did not drop it by name
# So it stays as 'age'
# (Double-check that your transformer doesn't rename or drop it)

# Step 3: Ensure the names match the actual shape
assert len(final_names) == X_fe.shape[1], f"Name count ({len(final_names)}) != features ({X_fe.shape[1]})"

# Step 4: Apply SelectKBest mask
selector = pipeline_kbest.named_steps['select']
mask = selector.get_support()

selected_feature_names = np.array(final_names)[mask]
print("Selected features:", selected_feature_names.tolist())


Selected features: ['age', 'serious_disease', 'fever', 'uses_corticosteroids', 'coping_strategy', 'nocturnal_pain', 'continuous_pain', 'duration_of_pain', 'paidwork', 'leg_pain']


In [ ]:
### re-run the pipeline with selected features

pipeline_kbest = ImbPipeline(steps=[
    ('imputer', IterativeImputer(max_iter=50, skip_complete=True)),
    #('smote', over),  # activate for oversampling
    ##('undersample', under),   # activate for undersampling
    ('scaler', StandardScaler()),
    ('select', SelectKBest(score_func=f_classif, k=20)),
    ('classifier', RandomForestClassifier(class_weight='balanced',n_estimators = 500, random_state = 42))
])

pipeline_kbest.fit(X_train, y_train)
score = pipeline_kbest.score(X_test, y_test)
print("Test score:", score)

Test score: 0.49673202614379086
